In [ ]:
%load_ext snakeviz
%load_ext tuna
import weave
import typing
from weave.tests import test_run_segment
from weave.weave_internal import define_fn
weave.capture_weave_server_logs()

In [ ]:
!rm -rf /tmp/local-artifacts
last_segment = test_run_segment.create_experiment(1000000, 10)

In [ ]:
experiment = last_segment.experiment()

group_key_name = "steppybin(pybinsequal (list (2, 500) , 2) )"

grouped = experiment.groupby(lambda row:
     weave.ops.dict_(**{group_key_name: 
         weave.ops.number_bin(
             in_=row['step'],
             bin_fn=weave.ops.numbers_bins_equal(
               weave.ops.list_.make_list(
                   **{"0": 2, "1": 500}),
               2))}))

def map_fn(row):
    return row.key().merge(
        weave.ops.dict_(
            **{
                "100": 100,
#                 "metric0 min": row.pick('metric0').min(),
#                 "metric0 max": row.pick('metric0').max(),
                "metric0 avg": row.pick("metric0").avg(),
                "circle": "circle",
            }
        )
    )
mapped = grouped.map(map_fn)
    
    
# mapped = grouped.map(lambda row:
#                     row.key().merge(
#                         weave.ops.dict_(
#                             **{
#                                 "100": 100,
#                                 #"step": row.pick("step"),
#                                 "metric0": row.pick("metric0").avg(),
#                                 #"string_col": row.pick("string_col"),
#                                 "circle": "circle",
#                             }
#                         )
#                     ))

#unnested = weave.ops.list_.unnest(mapped)
unnested = mapped.unnest()

listed = weave.ops.list_.make_list(**{"0": unnested})

In [ ]:
#weave.use(listed)

In [ ]:
import cProfile
cProfile.run("weave.use(listed)", 'profile_stats')

In [ ]:
# from pstats import SortKey
# import pstats
# p = pstats.Stats('profile_stats')
# p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats()
#p.strip_dirs().sort_stats(SortKey.TIME).print_stats()
#p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_callers('__iter__')